# The Lending Club

The Lending Club dataset is a collections of installment loan records, including credit bureau data (e.g.,
FICO, revolving balances, etc.) and loan performance data (e.g., loan status).

## Assumptions:

- Each row represents 1 borrower
- Each borrower has only 1 record/row
- Each borrower can have multiple loans/accounts, which is aggregated to the 1 row



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-pastel')
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("LoanStats_2015_subset.csv")

In [ ]:
df.head(10)

## Section A

### What is the monthly total loan volume in dollars and what is the monthly average loan size?

- `funded_amnt`: The total amount committed to that loan at that point in time.

- `loan_amnt`: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.


In [ ]:
df['issue_d'].value_counts()

In [ ]:
_months = {'Jan':'01', 'Feb':'02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12',}

def parse_dates(date_str):
    if isinstance(date_str, str):
        month_desc, year = date_str.split('-')
        month = _months[month_desc]
        return month
    else:
        return np.nan

In [ ]:
df['issue_month'] = df['issue_d'].apply(parse_dates)

In [ ]:
monthly_loan_amnts = df.groupby(by = 'issue_month')[['loan_amnt', 'funded_amnt']].sum()

In [ ]:
monthly_loan_amnts # loan amount applied = loan funded amount

In [ ]:
plt.figure(figsize = (7,5))
(monthly_loan_amnts['funded_amnt']/1e6).plot()
plt.title('Monthly Total Loan Volume (2015)')
plt.ylabel('$ Millions')
plt.xlabel('Month')

In [ ]:
monthly_avg_loan_amnts = df.groupby(by = 'issue_month')['funded_amnt'].mean()

In [ ]:
monthly_avg_loan_amnts

In [ ]:
plt.figure(figsize = (7,5))
(monthly_loan_amnts['funded_amnt']/1e6).plot()
plt.title('Monthly Average Loan Volume (2015)')
plt.ylabel('$ Millions')
plt.xlabel('Month')

### What are the default rates by Loan Grade?

- `grade`: LC assigned loan grade

- `loan_status`: Current status of the loan ('Fully Paid',
 'Current',
 'Charged Off',
 'Late (31-120 days)',
 'In Grace Period',
 'Late (16-30 days)',
 'Default')
 
 

In [ ]:
df['loan_status'].value_counts() # only 9 accounts are in default

In [ ]:
df['default_flag'] = (df['loan_status'] == 'Default') * 1

In [ ]:
default_by_grade = df.groupby(by = 'grade')['default_flag'].agg(['sum', 'mean']) # delinquency =  default

In [ ]:
# default frequency and rate by grade
default_by_grade

### Is Lending Club charging an appropriate interest rate for the risk?

- `term`: The number of payments on the loan. Values are in months and can be either 36 or 60

- `installment`: The monthly payment owed by the borrower if the loan originates (includes interest rate)

- `funded_amnt`: The total amount committed to that loan at that point in time

`total_expected_repayment`: `term` $\times$ `installment`

In [ ]:
df['term'] = df['term'].map({' 36 months': 36, ' 60 months':60}) # 3 years or 5 years

In [ ]:
total_payments = df['installment'] * df['term']
total_payments = total_payments.to_frame('total_payments')

In [ ]:
#pd.concat([total_payments, df['funded_amnt']], 1).head()

In [ ]:
df['total_expected_repayment'] = total_payments['total_payments'] * (1 - df['default_flag'])

In [ ]:
df['total_expected_repayment'].sum()

In [ ]:
df['funded_amnt'].sum()

In [ ]:
(df['total_expected_repayment'] - df['funded_amnt']).sum()

In [ ]:
df.groupby(by = 'grade')[['funded_amnt', 'total_expected_repayment']].sum() / 1e6 # divide by $1 Million

The total loan funded amount is `6,417,608,175` which is less than the expected repayment amount `8,309,102,650` so the Loan Lending Club is charging sufficient interest overall. Loans broken down by grade still present a profit regardlesss of risk of default.

## Section B - Modeling

### QA Data Set

Some basic issues with this data set include:

- missing values; some columns seem to be intended to be all blank or mostly blank. 

- missing values; However, other records seem to be missing at random (MAR)

- dates are in a human readable format, but not very "machine friendly". Parsing the dates takes some extra steps. Dates are in a non-standard format like YYYY-MM-DD

- some numerical values like interest rate have non-numeric characters in the values (i.e. 5.1% instead of 5.1). This again is a non-standard practice.

- zip code field is populated with zip codes ending in "xx"; however, this was probably done to anonymize the data

- employment title has duplicate characters resulting from human entered values. For example, "Teacher" and "teacher" are the same value however because not every character is upper (or lower) case, the data set assumes two different values.

- emp_length is a field recording number of years employed, however, this information is stored as categorical data instead of numerical values. The data is also censored after 10+ years.




In [ ]:
for c, s in df.items():
    if s.dtype == 'O':
        print(c)
        print(s.value_counts())

In [ ]:
df.isnull().mean()[df.isnull().mean() < 0.7]

In [ ]:
#df.isnull().sum()[df.isnull().mean() < 7]

In [ ]:
# the following columns seem to be mostly missing if not all missing
df.isnull().mean()[df.isnull().mean() > 0.7]

In [ ]:
#df['emp_length'].value_counts()

In [ ]:
#df['emp_title'].value_counts()

### Are there any glaringly erroneous data values?

In [ ]:
df['funded_amnt'].describe()

In [ ]:
def interst2float(int_rate):
    if isinstance(int_rate, str):
        int_rate = float(int_rate.replace('%', ''))
    return int_rate

In [ ]:
df['int_rate'] = df['int_rate'].apply(interst2float)

In [ ]:
def length_earliest_crline(date_str):
    if isinstance(date_str, str):
        year = date_str.split('-')[1]
        return 2015- int(year)
    else:
        return np.nan

In [ ]:
#df['earliest_cr_line'].apply(length_earliest_crline)
df['length_earliest_cr_line'] = df['earliest_cr_line'].apply(length_earliest_crline)

In [ ]:
df['loan_status'].value_counts().index

### Using any format and any modeling technique that you prefer, please create a model to predict default within the Lending Club dataset. Show any work that you would deem important in evaluating this process.

- There are 196,814 records with a `loan_status` = `Fully Paid`

- There are 9 records with a `loan_status` = `Default`

- All other `loan_status` will be ignored. Current and delinquent loans may possibly become default, therefore a good benchmark would be to pair `Fully Paid` vs `Default`.

- The data set imbalance is still severe even whislt dropping data points; this will hinder model performance


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.utils.multiclass import unique_labels
import patsy
from tqdm import tqdm
from tools import plot_confusion_matrix

In [ ]:
def fully_paid_or_default(status):
    return status in ['Default', 'Fully Paid']

In [ ]:
ind = df['loan_status'].apply(fully_paid_or_default)

In [ ]:
df = df.loc[ind].reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
with open("new_features.csv") as myfile:
    cols = list()
    for line in myfile:
        cols.append(line.strip())

In [ ]:
cols.remove('column')

In [ ]:
#cols
for c in cols:
    print(c, ":", df[c].dtype)

In [ ]:
for c in cols:
    if df[c].dtype == 'O':
        print(c, df[c].value_counts().shape[0])

In [ ]:
model_str = "default_flag ~ " + " + ".join(cols)

In [ ]:
model_str

In [ ]:
df['default_flag'].sum()

In [ ]:
#ind = df[cols].isnull().sum(axis=1) < 1 # choose columns with no missing data

In [ ]:
df['loan_status'].value_counts()

In [ ]:
Y, X = patsy.dmatrices(formula_like=model_str, data = df, return_type='dataframe')

In [ ]:
Y.shape

In [ ]:
X.shape

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, stratify=Y)

In [ ]:
train_x.shape

In [ ]:
train_y.sum()

In [ ]:
test_y.sum()

In [ ]:
ada_model = AdaBoostClassifier(n_estimators=1800)

In [ ]:
ada_model.fit(train_x, train_y['default_flag'])

In [ ]:
ada_preds = ada_model.predict(test_x)

In [ ]:
plot_confusion_matrix(test_y.astype(int), ada_preds.astype(int), classes=np.array([0,1]))

In [ ]:
y_score = ada_model.predict_proba(test_x)

In [ ]:
fpr, tpr, thres = roc_curve(test_y['default_flag'], y_score[:, 1])

In [ ]:
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize = (7, 5))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#0.62